# Training Resnet50 for Head's Position Detection
After preparing the `margined_cropped_images`, and `data.csv`, now we are able to feed them to a model to predict Hexbugs' heads' position.
The approach that we managed to work with was using ***transfer learning*** to build a better model. For this, we used pre-trained `Resnet50` model, which is trained on `CoCo` dataset. This helps us to extract feature better. Hoever, top layers of the `resnet` model are used for classification tasks. To modify the model to suit our problem, we removed tha last `Dense` layer and added two other `Dense` layers to build a regressor with only ***two*** outputs for `(x, y)`.
After modifying the model, we can train it over our dataset, and evaluate its performance on the validation set.
At last, we save the model for further use cases.

### Essential Imports

* `cv2` is required to read and write images.
* `json` is used to open `json` files.
* `random` is used to pick random samples for model evaluation.
* `logging` is used to ignore `warning` messages. (not mandatory)
* `numoy` is cruicial to work with images.
* `pandas` is used to read the `data.csv` file.
* `tensorflow` is used to create model. it's used as background for `Keras`.
* `sklearn` is used to split dataset to `train` and `test` sets.

In [1]:
import os
import cv2
import json
import random
import logging
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import ResNet101, DenseNet121, DenseNet169
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

tf.get_logger().setLevel(logging.ERROR)

2023-05-08 13:52:51.479250: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-08 13:52:51.580795: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 13:52:52.187996: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Reading data from `data.csv`

In [3]:
data = pd.read_csv('data.csv').drop(['Unnamed: 0'], axis=1).sort_values('ID', ascending=True)
data.head()

,CroppedHexBugCoordinationX,CroppedHexBugCoordinationY,OriginalBoxCoordinationX1,OriginalBoxCoordinationY1,Path,ID
4320,44,45,222,9,cropped_bugs/training018/frame0.jpg,0
1221,34,161,10,560,cropped_bugs/training045/frame0.jpg,0
2874,214,37,338,592,cropped_bugs/training058/frame0.jpg,0
1146,204,246,537,744,cropped_bugs/training020/frame0.jpg,0
2975,39,163,88,99,cropped_bugs/training022/frame0.jpg,0


In [12]:
# Load the images
images = []
y = []

# Iterates over the `data.csv` file's rows to read images and annotatopns.
for idx, row in data.iterrows():
#     img_path = f"{row.Path.replace('cropped_bugs', 'image_sharpning/sharpened_images')}"
    img_path = f"{row.Path}"
    annotations = [int(row.CroppedHexBugCoordinationX), int(row.CroppedHexBugCoordinationY)]
    # Reads the image
    img = load_img(img_path, color_mode='rgb')
    # Converts the image to a numpy array
    np_img = np.array(img)
    
#     extended_np_img = np.expand_dims(np_img, axis=2)
    # Adds the image to a list of images.
    images.append(np_img)
    # Adds the annotations to y.
    y.append(annotations)
    
X = np.array(images)
y = np.array(y)

print(X.shape)
print(y.shape)

(4321, 300, 300, 3)
(4321, 2)


In [13]:
# Splits the dataset to `train` and `test` sets with the given ratio.
# Also, shuffles the data to prevent feeding the model same data over different runs.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [14]:
print(f'X train shape: {X_train.shape}')
print(f'X test shape: {X_test.shape}')
print(f'y train shape: {y_train.shape}')
print(f'y test shape: {y_test.shape}')

X train shape: (3456, 300, 300, 3)
X test shape: (865, 300, 300, 3)
y train shape: (3456, 2)
y test shape: (865, 2)


In [15]:
# dropout_dense = 0.25

# dense_model = DenseNet121(
#     include_top=False,
#     weights="imagenet",
#     input_shape=X_train[0].shape)

# model = Sequential(dense_model, name='DenseNet169')

# model.add(layers.Dropout(dropout_dense))

# x = model.output
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
# x = Dense(64, activation='relu')(x)

# # The last layer should be the layer that computes the (x, y) coordinations.
# output_layer = Dense(2, activation='linear')(x)

# # Create the model based on the input and output layer.
# model = Model(inputs=model.input, outputs=output_layer)

In [16]:
# vgg_model = tf.keras.applications.VGG19(
#     include_top=False,
#     weights="imagenet",
#     input_shape=X_train[0].shape,
# )

# for layer in vgg_model.layers:
#     layer.trainable = True

# # Adds two dense layers for regression problem.
# # It basically uses the output of the `resnet50` model to feed the dense layers.
# x = vgg_model.output
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
# x = Dense(64, activation='relu')(x)

# # The last layer should be the layer that computes the (x, y) coordinations.
# output_layer = Dense(2, activation='linear')(x)

# # Create the model based on the input and output layer.
# model = Model(inputs=vgg_model.input, outputs=output_layer)

In [17]:
# model = Sequential()
# model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
# # model.add(Dropout(0.25))
# # model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))

# model.add(Conv2D(256, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))

# model.add(Conv2D(512, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# # model.add(MaxPooling2D((2, 2)))

# model.add(Conv2D(256, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))

# model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(16, activation='relu'))
# model.add(Dense(2))

In [18]:
# Creates a pre-trained model instance from class `ResNet50`.
# This model doesn't include the last classification dense layer.
# The given `input_shape` should be the same as our dataset images.
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(X_train[0].shape))

# Iterates over each layer in the model and makes them to be trainable.
for layer in resnet50.layers:
    layer.trainable = True

# Adds two dense layers for regression problem.
# It basically uses the output of the `resnet50` model to feed the dense layers.
x = resnet50.output
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)

# The last layer should be the layer that computes the (x, y) coordinations.
output_layer = Dense(2, activation='linear')(x)

# Create the model based on the input and output layer.
model = Model(inputs=resnet50.input, outputs=output_layer)

2023-05-08 13:56:19.307177: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-08 13:56:19.365381: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-08 13:56:19.365510: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

### Visualization
For better visualization, we use [tensorboard](https://www.tensorflow.org/tensorboard/get_started). It helps us to plot the figures real-time and analyze the model performance.

In [19]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# We need datetime to name our checkpoints.
import datetime

# Clear any logs from previous runs
!rm -rf ./logs/

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Creates a callback, which then will be called by the model during training process.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

/usr/bin/zsh: /home/farzam/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)


### Saving the best model
During the training process, we need to track the model's performance and make sure that at the end of the training process, we have the best model.
To do so, we used a custom `callback` that checks if the `val_loss` has been decreased over the past epoch. In that case, it will simply save the model to the provided directory, which can be used to load and evaluate later.
This call back is then called by the model during the training process.

In [20]:
# Specifies the checkpoint directory
checkpoint_filepath = 'checkpoint_callback'

# Creates an instance from the ModelCheckpoint class.
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [21]:
# Since we are dealing with regression problem, it's recommended to use `mse`.
# Also we use `Adam` as our optimizer.
model.compile(loss='mse', optimizer='adam')

In [22]:
# Prints a summary of the compiled model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 306, 306, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 150, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 75, 75, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 75, 75, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 75, 75, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 75, 75, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 38, 38, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 38, 38, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 38, 38, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 38, 38, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 19, 19, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 19, 19, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 19, 19, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 19, 19, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 19, 19, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 19, 19, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 19, 19, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 19, 19, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                                                  
 conv5_block1_add (Add)         (None, 10, 10, 2048  0           ['conv5_block1_0_bn[0][0]',      
                                )                                 'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 10, 10, 2048  0           ['conv5_block1_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 10, 10, 512)  1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 10, 10, 512)  2048       ['conv5_block2_1_conv[0][0]']    
 ization) 

### Training the model
Now that we have everything ready, we can train the model. we pass the `validation_data` containing the `X_test` and `y_test` to the model as well.
Also, for above mentioned checkopints, we need to call those callbacks.

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=8, validation_data=(X_test, y_test), callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/20


2023-05-08 13:57:18.700094: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-08 13:57:20.383455: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-08 13:57:20.388857: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7ff0d9e8f530 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-08 13:57:20.388875: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-05-08 13:57:20.402728: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-08 13:57:20.544354: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifeti

432/432 [==============================] - ETA: 0s - loss: 21458.7402

432/432 [==============================] - 88s 152ms/step - loss: 21458.7402 - val_loss: 468216.9062
Epoch 2/20
432/432 [==============================] - ETA: 0s - loss: 18090.8672

432/432 [==============================] - 63s 147ms/step - loss: 18090.8672 - val_loss: 15959.7861
Epoch 3/20
 58/432 [===>..........................] - ETA: 40s - loss: 16638.1016

In [ ]:
# Plotting the tensorboard data in jupyter notebook
%tensorboard --logdir logs/fit

In [ ]:
def compute_error(y, y_pred):
    """
    Since the competion computes the error based on Euclidian distances, this function computes
    the error for predicted and ground truth results.
    
    @param: list y. a (1, 2) list with original coordinations.
    @param: list y_pred. a (1, 2) list containing the predicted coordinations.
    
    @return: float error. The Euclidian distance between these two given datapoints.
    """
    error = np.sqrt(np.abs(y[0] - y_pred[0]) ** 2 + np.abs(y[1] - y_pred[1]) ** 2)
    return error

In [ ]:
def compute_original_coordination(predicted_coordination, original_box_coordination):
    """
    Computes the coordinations of the predicted Hexbug's head's coordination in the original image
    using the bounding box coordinations.
    
    @param: list predicted_coordination. a (1, 2) list with (x, y) format.
    @param: list original_box_coordination. a (1, 2) list with (x, y) format of bounding box.
    
    @return: list. Coordinations of the predicted Hexbug's head's coordination in the original image
    """
    return (predicted_coordination[0] + original_box_coordination[0], predicted_coordination[1] + original_box_coordination[1])

### Loading the best model

In [ ]:
reconstructed_model = tf.keras.models.load_model("checkpoint_callback")
# reconstructed_model = tf.keras.models.load_model("resnet50_trained_model_data/v1.0/checkpoint_callback")

### Sample Evaluation
Now that we have successfully trained the model, let's see its performance on unseen data.

In [ ]:
num_samples = 10

# Chooses random indices
random_samples_indices = random.sample(range(0, len(X_test)), num_samples)

# Iterates over each index and retrieves the image and annotaion
for sample_index in random_samples_indices:
    sample_img = X_test[sample_index]
    
    # Configs for drawing a circle on the image.
    center = y_test[sample_index]
    radius = 5
    # Color map is BGR
    color = (0, 255, 0)
    color_pred = (0, 0, 255)
    thickness = 5
    
    # Since we predict only one image at a time, we need to expand its dimention
    # to fit the input layer of our model.
    expanded_img = np.expand_dims(sample_img, axis=0)
    
    # Predicts the coordinations
    predicted = list(map(int, reconstructed_model.predict(expanded_img)[0]))
    
    # Computes the error
    error = round(compute_error(center, predicted), 3)
    
    print(f'y_predicted: {predicted}, y: {center}, Error: {error}')
    
    # Draws a circle centered in the correct coordination.
    img_lv1 = cv2.circle(sample_img, center, radius, color, thickness)
    # Draws a circle centered in the predicted coordination.
    img_lv2 = cv2.circle(img_lv1, predicted, radius, color_pred, thickness)
    
    # Shows the output image.
    cv2.imshow(f'Sample No. {sample_index}, error: {error}', img_lv2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### Sample for coordination convertion

In [ ]:
num_samples = 10

random_samples_indices = random.sample(range(0, len(data)), num_samples)

for sample_id in random_samples_indices:
    sample_img_path = data.iloc[sample_id].Path

    # Constructs the path to the original frame
    original_image_path = f"Videos/{sample_img_path.split('/')[1]}/{sample_img_path.split('/')[2]}"

    sample_img = np.array(load_img(sample_img_path))

    expanded_img = np.expand_dims(sample_img, axis=0)

    predicted = list(map(int, reconstructed_model.predict(expanded_img)[0]))

    radius = 10
    color = (0, 255, 0)
    thickness = -1

    # Computes the coordination of the Hexbug's head in the original image.
    scaled_coordinations = compute_original_coordination(predicted, (data.iloc[sample_id].OriginalBoxCoordinationX1, data.iloc[sample_id].OriginalBoxCoordinationY1))

    # Reads and converts the original image into numpy array.
    original_sample_img = np.array(load_img(original_image_path))

    annotated_img = cv2.circle(sample_img, predicted, radius, color, thickness)

    cv2.circle(original_sample_img, scaled_coordinations, radius, color, thickness)
    
    center = [data.iloc[sample_id].CroppedHexBugCoordinationX, data.iloc[sample_id].CroppedHexBugCoordinationY]
    
    error = round(compute_error(center, predicted), 3)
    
    file_name = f'Sample ID: {sample_id}\nTruth: {center}, Predicted: {predicted}, Error: {error}'
    
    print(file_name)
    print('------------------------------------------------------------------------')
    
    if not os.path.exists('annotated_images'):
        os.mkdir('annotated_images')
    
    cv2.imshow(f'Sample ID. {sample_id}', original_sample_img)
    cv2.imwrite(f'annotated_images/{file_name}.jpg', original_sample_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# reconstructed_model.summary()

In [ ]:
# model = Sequential()
# model.add(Conv2D(64, (3, 3), activation='relu', input_shape=X_train[0].shape))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(512, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(1028, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(256, (3, 3), activation='relu', kernel_regularizer='L2'))
# # model.add(Dropout(0.25))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(2))

In [ ]:
# model.summary()

In [ ]:
# reconstructed_model.compile(loss='mse', optimizer='adam')

In [ ]:
# history = reconstructed_model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), callbacks=[tensorboard_callback, model_checkpoint_callback])